In [1]:
from __future__ import unicode_literals, print_function
import json
import pathlib
import random

import spacy
from spacy.pipeline import EntityRecognizer
from spacy.gold import GoldParse
from spacy.tagger import Tagger

import os
import re
 
try:
    unicode
except:
    unicode = str

In [2]:
nlp = spacy.load('en')
#nlp = spacy.load('en', parser=False, entity=False, add_vectors=False)

In [3]:
def tex2doc(tex_file):  #read the whole tex file in the spaCy doc object
    with open(tex_file, 'r') as tex:
        data=tex.read()
    doc = nlp(data)
    return doc

In [4]:
def rule_based_annotation(doc):
    annotation = []
    for match in re.finditer('let \$(\S+( \S+){0,3})\$ be an? (\S+)', doc.text, re.IGNORECASE):
        annotation.append((match.span(1)[0],match.span(1)[1], 'VAR'))
        annotation.append((match.span(3)[0],match.span(3)[1], 'TYPE'))    
    return (doc.text, annotation)

In [5]:
annotated_data=[]

directory = os.fsencode('tex_files/')
for file in os.listdir(directory)[0:3]:
    filename = os.fsdecode(file)
    print("file: ", filename)
    doc = tex2doc(os.path.join(os.fsdecode(directory), filename))
    annotated_data.append(rule_based_annotation(doc))   
    
    

file:  pione.tex
file:  intersection.tex
file:  spaces-simplicial.tex


In [6]:
print(len(annotated_data))
print(annotated_data[0])

3
("\\input{preamble}\n\n% OK, start here.\n%\n\\begin{document}\n\n\\title{Fundamental Groups of Schemes}\n\n\n\\maketitle\n\n\\phantomsection\n\\label{section-phantom}\n\n\\tableofcontents\n\n\\section{Introduction}\n\\label{section-introduction}\n\n\\noindent\nIn this chapter we discuss Grothendieck's fundamental group of a scheme\nand applications. A foundational reference is \\cite{SGA1}.\nA nice introduction is \\cite{Lenstra}.\nOther references \\cite{Murre-lectures} and \\cite{Grothendieck-Murre}.\n\n\n\n\n\n\n\n\n\n\n\\section{Schemes \\'etale over a point}\n\\label{section-schemes-etale-point}\n\n\\noindent\nIn this section we describe schemes \\'etale over the spectrum of a field.\nBefore we state the result we introduce the category of $G$-sets for a\ntopological group $G$.\n\n\\begin{definition}\n\\label{definition-G-set-continuous}\nLet $G$ be a topological group.\nA {\\it $G$-set}, sometime called a {\\it discrete $G$-set},\nis a set $X$ endowed with a left action $a : G

In [7]:
random.shuffle(annotated_data)
train_data = annotated_data[:-1]
test_data = annotated_data[-1:] #we hold out one tex file for testing

In [8]:
def train_ner(nlp, train_data, entity_types):
    # Add new words to vocab.
    for raw_text, _ in train_data:
        doc = nlp.make_doc(raw_text)
        for word in doc:
            _ = nlp.vocab[word.orth]

    # Train NER.
    ner = EntityRecognizer(nlp.vocab, entity_types=entity_types)
    for itn in range(5):
        random.shuffle(train_data)
        for raw_text, entity_offsets in train_data:
            doc = nlp.make_doc(raw_text)
            gold = GoldParse(doc, entities=entity_offsets)
            ner.update(doc, gold)
    return ner

In [9]:
ner = train_ner(nlp, train_data, ['VAR', 'TYPE'])

In [10]:
#first test on a simple sentence
doc = nlp.make_doc('Let $S$ be a scheme.')  
nlp.tagger(doc)
ner(doc)
for word in doc:
    print(word.text, word.ent_type_)

Let 
$ 
S$ 
be 
a 
scheme TYPE
. TYPE


In [11]:
#then test on the hold out tex file; there are no 'VAR' tags detected, maybe something to do with dollar sign tokenization...
doc = nlp.make_doc(test_data[0][0])  
nlp.tagger(doc)
ner(doc)
for word in doc
    print(word.text, "\t" + word.ent_type_)

\input{preamble 	
} 	


 	
% 	
OK 	
, 	
start 	
here 	
. 	

 	
% 	

 	
\begin{document 	
} 	


 	
\title{Simplicial 	
Spaces 	
} 	



 	
\maketitle 	


 	
\phantomsection 	

 	
\label{section 	
- 	
phantom 	
} 	


 	
\tableofcontents 	


 	
\section{Introduction 	
} 	

 	
\label{section 	
- 	
introduction 	
} 	


 	
\noindent 	

 	
This 	
chapter 	
develops 	
some 	
theory 	
concerning 	
simplicial 	
topological 	
spaces 	
, 	

 	
simplicial 	
ringed 	
spaces 	
, 	
simplicial 	
schemes 	
, 	
and 	
simplicial 	
algebraic 	
spaces 	
. 	

 	
The 	
theory 	
of 	
simplicial 	
spaces 	
sometimes 	
allows 	
one 	
to 	
prove 	
local 	
to 	
global 	

 	
principles 	
which 	
appear 	
difficult 	
to 	
prove 	
in 	
other 	
ways 	
. 	

 	
Some 	
example 	
applications 	
can 	
be 	
found 	
in 	
the 	
papers 	

 	
\cite{faltings_finiteness 	
} 	
, 	
\cite{Kiehl 	
} 	
, 	
and 	
\cite{HodgeIII}. 	


 	
\medskip\noindent 	

 	
We 	
assume 	
throughout 	
that 	
the 	
reader 	
is 	
familiar 	
with 	
the 	

$ 	
g_{n!}$ 	
is 	
exact 	
. 	

 	
\end{enumerate 	
} 	

 	
\end{lemma 	
} 	


 	
\begin{proof 	
} 	

 	
Besides 	
the 	
properties 	
of 	
our 	
functor 	
mentioned 	
in 	
the 	
statement 	
, 	

 	
the 	
category 	
$ 	
X_{n 	
, 	
Zar}$ 	
has 	
fibre 	
products 	
and 	
equalizers 	

 	
and 	
the 	
functor 	
commutes 	
with 	
them 	
( 	
beware 	
that 	
$ 	
X_{Zar}$ 	
does 	
not 	

 	
have 	
all 	
fibre 	
products 	
) 	
. 	
Hence 	
the 	
lemma 	
follows 	
from 	
the 	
discussion 	
in 	

 	
Sites 	
, 	
Sections 	
\ref{sites 	
- 	
section 	
- 	
cocontinuous 	
- 	
functors 	
} 	
and 	

 	
\ref{sites 	
- 	
section 	
- 	
cocontinuous 	
- 	
morphism 	
- 	
topoi 	
} 	

 	
and 	

 	
Modules 	
on 	
Sites 	
, 	
Section 	
\ref{sites 	
- 	
modules 	
- 	
section 	
- 	
exactness 	
- 	
lower 	
- 	
shriek}. 	

 	
More 	
precisely 	
, 	

 	
Sites 	
, 	
Lemmas 	
\ref{sites 	
- 	
lemma 	
- 	
cocontinuous 	
- 	
morphism 	
- 	
topoi 	
} 	
, 	

 	
\ref{sites 	
- 	
lemma 	
- 	
when 	
- 	
shriek 	
} 	
, 	
and 	


equivalent 	
to 	
$ 	
\Delta[0]$ 	
, 	
see 	

 	
Simplicial 	
, 	
Example 	
\ref{simplicial 	
- 	
example 	
- 	
simplex 	
- 	
contractible 	
} 	
, 	

 	
and 	
since 	
`` 	
taking 	
free 	
abelian 	
groups 	
'' 	
is 	
a 	
functor 	
, 	

 	
we 	
see 	
that 	
the 	
complex 	
above 	
is 	
homotopy 	
equivalent 	
to 	

 	
the 	
free 	
abelian 	
group 	
on 	
$ 	
\Delta[0]$ 	

 	
( 	
Simplicial 	
, 	
Remark 	
\ref{simplicial 	
- 	
remark 	
- 	
homotopy 	
- 	
better 	
} 	
and 	

 	
Lemma 	
\ref{simplicial 	
- 	
lemma 	
- 	
homotopy 	
- 	
equivalence 	
- 	
s 	
- 	
N 	
} 	
) 	
. 	

 	
This 	
complex 	
is 	
acyclic 	
in 	
positive 	
degrees 	

 	
and 	
equal 	
to 	
$ 	
\mathbf{Z}$ 	
in 	
degree 	
$ 	
0$. 	

 	
\end{proof 	
} 	


 	
\begin{lemma 	
} 	

 	
\label{lemma 	
- 	
simplicial 	
- 	
sheaf 	
- 	
cohomology 	
} 	

 	
Let 	
$ 	
X$ 	
be 	
a 	
simplicial 	TYPE
topological 	
space 	
. 	
Let 	
$ 	
\mathcal{F}$ 	
be 	
an 	
abelian 	TYPE

 	
sheaf 	
on 	
$ 	
X$. 	
There 	
is 	
a 	
spectral 	
sequen

object 	
of 	
$ 	
\mathcal{C}_{total}$ 	
is 	
an 	
object 	
$ 	
U$ 	
of 	

 	
$ 	
\mathcal{C}_n$ 	
for 	
some 	
$ 	
n$ 	
, 	

 	
\item 	
a 	
morphism 	
$ 	
( 	
\varphi 	
, 	
f 	
) 	
: 	
U 	
\to 	
V$ 	
of 	
$ 	
\mathcal{C}_{total}$ 	

 	
is 	
given 	
by 	
a 	
map 	
$ 	
\varphi 	
: 	
[ 	
m 	
] 	
\to 	
[ 	
n]$ 	
with 	

 	
$ 	
U 	
\in 	
\Ob(\mathcal{C}_n)$ 	
, 	
$ 	
V 	
\in 	
\Ob(\mathcal{C}_m)$ 	

 	
and 	
a 	
morphism 	
$ 	
f 	
: 	
u_\varphi(U 	
) 	
\to 	
V$ 	
of 	
$ 	
\mathcal{C}_m$ 	
, 	
and 	

 	
\item 	
a 	
covering 	
$ 	
\{(\text{id 	
} 	
, 	
f_i 	
) 	
: 	
  	
U_i 	
\to 	
U\}$ 	
in 	
$ 	
\mathcal{C}_{total}$ 	

 	
is 	
given 	
by 	
an 	
$ 	
n$ 	
and 	
a 	
covering 	
$ 	
\{f_i 	
: 	
U_i 	
\to 	
U\}$ 	

 	
of 	
$ 	
\mathcal{C}_n$. 	

 	
\end{enumerate 	
} 	

 	
\end{lemma 	
} 	


 	
\begin{proof 	
} 	

 	
Composition 	
of 	
$ 	
( 	
\varphi 	
, 	
f 	
) 	
: 	
U 	
\to 	
V$ 	
with 	
$ 	
( 	
\psi 	
, 	
g 	
) 	
: 	
V 	
\to 	
W$ 	

 	
is 	
given 	
by 	
$ 	
( 	
\varphi 	
\circ 	
\psi 	
, 	
g

} 	


 	
\begin{proof 	
} 	

 	
The 	
first 	
statement 	
follows 	
from 	

 	
Homology 	
, 	
Lemma 	
\ref{homology 	
- 	
lemma 	
- 	
adjoint 	
- 	
preserve 	
- 	
injectives 	
} 	

 	
and 	

 	
Lemma 	
\ref{lemma 	
- 	
restriction 	
- 	
to 	
- 	
components 	
- 	
site}. 	

 	
The 	
second 	
statement 	
from 	

 	
Derived 	
Categories 	
, 	
Lemma 	
\ref{derived 	
- 	
lemma 	
- 	
adjoint 	
- 	
preserve 	
- 	
K 	
- 	
injectives 	
} 	

 	
and 	

 	
Lemma 	
\ref{lemma 	
- 	
restriction 	
- 	
to 	
- 	
components 	
- 	
site}. 	

 	
\end{proof 	
} 	








 	
\section{Augmentations 	
of 	
simplicial 	
sites 	
} 	

 	
\label{section 	
- 	
augmentation 	
- 	
simplicial 	
- 	
sites 	
} 	


 	
\noindent 	

 	
We 	
continue 	
in 	
the 	
fashion 	
described 	
in 	

 	
Section 	
\ref{section 	
- 	
simplicial 	
- 	
sites 	
} 	

 	
working 	
out 	
the 	
meaning 	
of 	
augmentations 	
in 	
cases 	
A 	
and 	
B 	

 	
treated 	
in 	
that 	
section 	
. 	


 	
\begin{remark 	
} 	

 	
\label{remark 	
- 	
augm

...

a 	
) 	
a 	
flat 	
, 	
locally 	
finitely 	
presented 	
, 	
surjective 	
map 	
defines 	

 	
an 	
fppf 	
covering 	
, 	
and 	

 	
( 	
b 	
) 	
fppf 	
cohomology 	
of 	
sheaves 	
pulled 	
back 	
from 	
the 	
small 	
\'etale 	
site 	

 	
agrees 	
with 	
\'etale 	
cohomology 	
as 	
we 	
have 	
seen 	
in 	

 	
More 	
on 	
Cohomology 	
of 	
Spaces 	
, 	
Section 	

 	
\ref{spaces 	
- 	
more 	
- 	
cohomology 	
- 	
section 	
- 	
fppf 	
- 	
etale}. 	


 	
\begin{lemma 	
} 	

 	
\label{lemma 	
- 	
compare 	
- 	
simplicial 	
- 	
objects 	
- 	
fppf 	
- 	
etale 	
} 	

 	
Let 	
$ 	
S$ 	
be 	
a 	
scheme 	TYPE
. 	TYPE
Let 	
$ 	
X$ 	
be 	
an 	
algebraic 	TYPE
space 	
over 	
$ 	
S$. 	

 	
Let 	
$ 	
U$ 	
be 	
a 	
simplicial 	TYPE
algebraic 	
space 	
over 	
$ 	
S$. 	

 	
Let 	
$ 	
a 	
: 	
U 	
\to 	
X$ 	
be 	
an 	
augmentation 	TYPE
. 	
There 	
is 	
a 	
commutative 	
diagram 	

 	
$ 	
$ 	

 	
\xymatrix 	
{ 	

 	
\Sh((\textit{Spaces}/U)_{fppf 	
, 	
total 	
} 	
) 	
\ar[r]_-h 	
\ar[d]_{a_{fppf 	
} 	
} 	
& 	

 

S$. 	

 	
Let 	
$ 	
U$ 	
be 	
a 	
simplicial 	TYPE
algebraic 	
space 	
over 	
$ 	
S$. 	
Let 	
$ 	
a 	
: 	
U 	
\to 	
X$ 	

 	
be 	
an 	
augmentation 	TYPE
. 	
Let 	
$ 	
\mathcal{F}$ 	
be 	
an 	
abelian 	TYPE
sheaf 	

 	
on 	
$ 	
X_\etale$. 	
Let 	
$ 	
\mathcal{F}_n$ 	
be 	
the 	
pullback 	
to 	
$ 	
U_{n 	
, 	
\etale}$. 	

 	
If 	
$ 	
U$ 	
is 	
an 	
fppf 	
hypercovering 	
of 	
$ 	
X$ 	
, 	
then 	

 	
there 	
exists 	
a 	
canonical 	
spectral 	
sequence 	

 	
$ 	
$ 	

 	
E_1^{p 	
, 	
q 	
} 	
= 	
H^q_\etale(U_p 	
, 	
\mathcal{F}_p 	
) 	

 	
$ 	
$ 	

 	
converging 	
to 	
$ 	
H^{p 	
+ 	
q}_\etale(X 	
, 	
\mathcal{F})$. 	

 	
\end{lemma 	
} 	


 	
\begin{proof 	
} 	

 	
Immediate 	
consequence 	
of 	
Lemmas 	
\ref{lemma 	
- 	
compute 	
- 	
via 	
- 	
fppf 	
- 	
hypercovering 	
} 	

 	
and 	
\ref{lemma 	
- 	
simplicial 	
- 	
sheaf 	
- 	
cohomology 	
- 	
site}. 	

 	
\end{proof 	
} 	









 	
\section{Fppf 	
hypercoverings 	
of 	
algebraic 	
spaces 	
: 	
modules 	
} 	

 	
\label{section 	
- 	

\to 	
\Sh(X_\etale)$ 	

 	
is 	
as 	
in 	
Section 	
\ref{section 	
- 	
simplicial 	
- 	
algebraic 	
- 	
spaces}. 	

 	
\end{lemma 	
} 	


 	
\begin{proof 	
} 	

 	
First 	
observe 	
that 	
the 	
maps 	
$ 	
a_n 	
: 	
U_n 	
\to 	
X$ 	
and 	
$ 	
d^n_i 	
: 	
U_n 	
\to 	
U_{n 	
- 	
1}$ 	

 	
are 	
flat 	
, 	
locally 	
of 	
finite 	
presentation 	
, 	
and 	
surjective 	
by 	

 	
Hypercoverings 	
, 	
Remark 	
\ref{hypercovering 	
- 	
remark 	
- 	
P 	
- 	
covering}. 	


 	
\medskip\noindent 	

 	
Recall 	
that 	
an 	
$ 	
\mathcal{O}_U$-module 	
$ 	
\mathcal{F}$ 	
is 	
quasi 	
- 	
coherent 	
if 	
and 	

 	
only 	
if 	
it 	
is 	
cartesian 	
and 	
$ 	
\mathcal{F}_n$ 	
is 	
quasi 	
- 	
coherent 	
for 	
all 	
$ 	
n$. 	

 	
See 	
Lemma 	
\ref{lemma 	
- 	
quasi 	
- 	
coherent 	
- 	
sheaf}. 	

 	
By 	
Lemma 	
\ref{lemma 	
- 	
Serre 	
- 	
subcat 	
- 	
cartesian 	
- 	
modules 	
} 	

 	
( 	
and 	
flatness 	
of 	
the 	
maps 	
$ 	
d^n_i 	
: 	
U_n 	
\to 	
U_{n 	
- 	
1}$ 	
shown 	
above 	
) 	

 	
the 	
carte

: 	
[ 	
n 	
] 	
\to 	
[ 	
m]$ 	
we 	
define 	

 	
$ 	
K_\varphi 	
: 	
L(f_\varphi)^*K_n 	
\to 	
K_m$ 	

 	
using 	

 	
$ 	
$ 	

 	
\xymatrix 	
{ 	

 	
L(f_\varphi)^*K_n 	
\ar@{=}[r 	
] 	
& 	

 	
L(f_\varphi)^ 	
* 	
L(f_{\tau^n_n})^*K_0 	
\ar@{=}[r 	
] 	
& 	

 	
L(f_{\tau^m_{\varphi(n)}})^*K_0 	
\ar@{=}[r 	
] 	
& 	

 	
L(f_{\tau^m_{\varphi(n)m}})^ 	
* 	
L(f_{\delta^1_1})^*K_0 	

 	
\ar[d]_{L(f_{\tau^m_{\varphi(n)m}})^*\alpha 	
} 	
\\ 	

 	
& 	

 	
K_m 	
\ar@{=}[r 	
] 	
& 	

 	
L(f_{\tau^m_m})^*K_0 	
\ar@{=}[r 	
] 	
& 	

 	
L(f_{\tau^m_{\varphi(n)m}})^ 	
* 	
L(f_{\delta^1_0})^*K_0 	

 	
} 	

 	
$ 	
$ 	

 	
We 	
omit 	
the 	
verification 	
that 	
the 	
cocycle 	
condition 	

 	
implies 	
the 	
maps 	
compose 	
correctly 	
( 	
in 	
their 	
respective 	

 	
derived 	
categories 	
) 	
and 	
hence 	
give 	
rise 	
to 	
a 	

 	
simplicial 	
systems 	
of 	
the 	
derived 	
category 	
of 	
modules\footnote{This 	

 	
verification 	
is 	
the 	
same 	
as 	
that 	
done 	
in 	
the 	
proof 	

 	
of 	
L

- 	
cohomology 	
- 	
lemma 	
- 	
cohomological 	
- 	
descent 	
- 	
etale 	
- 	
ph}. 	

 	
Hence 	
$ 	
Rh_*h^{-1}$ 	
is 	
the 	
identity 	
functor 	
on 	

 	
$ 	
D^+(U_\etale)$ 	
by 	

 	
Lemma 	
\ref{lemma 	
- 	
direct 	
- 	
image 	
- 	
morphism 	
- 	
simplicial 	
- 	
sites}. 	

 	
We 	
have 	

 	
\begin{align 	
* 	
} 	

 	
Ra_*(a^{-1}K 	
) 	

 	
& 	
= 	

 	
Ra_*Rh_*h^{-1}a^{-1}K 	
\\ 	

 	
& 	
= 	

 	
Rh_{-1 	
, 	
* 	
} 	
Ra_{ph 	
, 	
* 	
} 	
a_{ph}^{-1}(h_{-1})^{-1}K 	
\\ 	

 	
& 	
= 	

 	
Rh_{-1 	
, 	
* 	
} 	
( 	
h_{-1})^{-1}K 	
\\ 	

 	
& 	
= 	

 	
K 	

 	
\end{align 	
* 	
} 	

 	
The 	
first 	
equality 	
by 	
the 	
discussion 	
above 	
, 	
the 	
second 	
equality 	

 	
because 	
of 	
the 	
commutativity 	
of 	
the 	
diagram 	
in 	

 	
Lemma 	
\ref{lemma 	
- 	
compare 	
- 	
simplicial 	
- 	
objects 	
} 	
, 	
the 	
third 	
equality 	
by 	

 	
Lemma 	
\ref{lemma 	
- 	
hypercovering 	
- 	
X 	
- 	
simple 	
- 	
descent 	
- 	
bounded 	
- 	
abelian 	
} 	

 	
as 	
$ 	
U$ 	
is 	
a 	
hyperco